# Problem statement

This Kernel is an attempt to find some clues to generalized problem, that can give solution for this competition. New problem formulation is:
* Define maximum "ancestral" depth (MAD) as maximum number of turns we can go backwards in time in game of life among all possible parent states
* Find algorithm that for any board state will return one of the immediate parent states with maximum "ancestral" depth

So all states that don't have parent state will have MAD = 0.
Then all states that have parents only of MAD=0, will have MAD=1. And so on..

For example if some board state has grandparents but not grand-grandparents (MAD=2), hypotetical algorithm will return any parent state that also has parent.

We know that all "stop" states in the train and test datasets have MAD of at least 6-10 ("delta(1-5) + warmup(5)").
So algorithm that returns parent state with maximum MAD among all parents and does so for any initial state, will guarantee, that it can be applied recursively at least "delta" times, giving us at least one of desired "start" states.
Let's call it Algorithm of God for now ;) 

Here is one statement valid for any finite board, that could give a clue for desired algorithm:
* Any initial state has MAD="infinity", if and only if there is a cycle of states that includes initial state
(I'll leave proof of this statement to the Reader)

Let's try to find more patterns or properties like the one above. 


# Reduce board size

In attempt to find patterns that can help formulate "Algorithm of God", I decided to reduce board size to be fully solvable in memory and still meaningful.
* 1x1 the only cell is own neighbour 8 times
* 2x2 cells have same other cells as 2 or 4 of their neighbours
* 3x3 Every cell is a neighbour of any other cell
* 3x4 Cells in each row share all same neighbours

So, 4x4 is a first board size that starts to show some properties that possibly can be generalized to larger boards.
You can find complete research on it in my previous Kernel:
https://www.kaggle.com/elvenmonk/complete-4x4-board-analysis

Next size to try is 5x5. It is significantly larger and longer to compute, so I had to optimize and cache "hash" function before moving on.

**Note on hashing function:**

There are 2^25 possible states of 5x5 board. We can code each state as a single number in range 0-33554431.
To further reduce number of dirrerent states we will consider states equal if we can get one state from another by translation (E.g. using "numpy.roll"), rotation or flip around any axis

In [ ]:
import numpy as np
from scipy.signal import convolve2d
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import networkx as nx

SIZE = 5
SIZE2 = SIZE**2
M = 2**SIZE
N = 2**SIZE2
NM = N//M

empty_board = np.zeros((SIZE,SIZE), dtype=bool)
m2i = np.array([2**i for i in range(SIZE2)]).reshape(SIZE,SIZE)
window = np.ones((3, 3))

def plot_3d(solution_3d: np.ndarray, title, size=3, max_cols=2*SIZE, has_mad=False, tile=(1,1), vmin=0, vmax=1):
    N = len(solution_3d)
    if N <= 0:
        return
    cols = min(N, max_cols)
    rows = (N - 1) // max_cols + 1
    plt.figure(figsize=(cols*size, rows*size))
    if not has_mad:
        plt.suptitle(title)
    for t in range(N):
        board = np.tile(solution_3d[t], tile)
        plt.subplot(rows, cols, t + 1)
        plt.imshow(board, cmap='binary', vmin=vmin, vmax=vmax)
        if has_mad:
            plt.title(f'MAD {t}')
    plt.show()

def life_step(X: np.ndarray):
    Y = convolve2d(X, window, mode='same', boundary='wrap')
    return (Y == 3) | (X & (Y == 4)), Y

def print_base(a):
    print(np.vectorize(lambda x: x[3:])(np.vectorize(hex)(a+N)))#, base=M)))

In [ ]:
r = np.arange(SIZE)[:, None]
c = SIZE*r
rr = np.cumsum(2**r//2)[:, None] # 0,1,3,7
cc = np.cumsum(2**c//M)[:, None] # 0, 1, 17, 273
print_base(np.hstack([r, c, rr, cc]))

Sr = np.sum(2**r)
Sc = np.sum(2**c)
Sd = sum(1 << (SIZE - 1) * (i + 1) for i in range(SIZE))
print(hex(Sr), hex(Sc), hex(Sd))

Scr = Sc * 2**r
Src = Sr * 2**c

Srcc = Sr * cc
Scrr = Sc * rr
ScccSr = (Sc - cc) * Sr
SrrrSc = (Sr - rr) * Sc
print_base(np.hstack([Scr, Src, Srcc, Scrr, ScccSr, SrrrSc]))

def h_flip(x):
    return np.sum((x & Scr) << (SIZE - 1) >> 2*r, axis=0)

def v_flip(x):
    return np.sum((x & Src) << (SIZE2 - SIZE) >> 2*c, axis=0)

def d_flip(x):
    y = x & Sd
    for i in range(1,SIZE):
        y |= ((x & (Sd >> (i * SIZE))) << (i * (SIZE + 1))) | ((x & (Sd << (i * SIZE))) >> (i * (SIZE + 1)))
    return y

def get_hash(n):
    h = h_flip(n)
    v = v_flip(n)
    hv = h_flip(v)
    n = np.vstack([n, h, v, hv, d_flip(n), d_flip(h), d_flip(v), d_flip(hv)])[:,None,None,:]
    n = ((n & Srcc) << (SIZE2 - c)) + ((n & ScccSr) >> c)
    n = n.T
    n = ((n & Scrr) << (SIZE - r)) + ((n & SrrrSc) >> r)
    return np.min(n, axis=(1, 2, 3))

Every state in Hash function has 5x5x8 equivalent states (various translations, rotations and flips). So calculations on entire numpy array don't fit in memory, thus we have to split them into parts:

In [ ]:
n = np.arange(N)
h = np.arange(N)
for i in tqdm(range(M), total=M):
    h[i*N//M:(i+1)*N//M] = get_hash(n[i*N//M:(i+1)*N//M])

# NetworkX DiGraph

Transitions from previous to next board state can be represented as directed graph. Building NetworkX graph will allow us to utilize its various algorithms to analyze and traverse this graph.

In [ ]:
board = (m2i & n[:,None,None]) != 0
conv_board = np.zeros(board.shape, dtype=int)
G = nx.DiGraph()
for n0 in tqdm(range(N), total=N, mininterval=1):
    if G.has_node(h[n0]) and len(list(G.successors(h[n0]))) > 0:
        continue
    next_board, conv_board[n0] = life_step(board[n0])
    n1 = np.sum(next_board * m2i)
    G.add_edge(h[n0], h[n1])
n, h = None, None
print(nx.info(G))

First let's extract all cycles. Corresponding board states are the only ones to have infinite MAD. 

In [ ]:
cyclic = [c for cycle in nx.simple_cycles(G) for c in cycle]
for node in cyclic:
    G.remove_node(node)
plot_3d(board[cyclic], f'Cyclic boards')
plot_3d(conv_board[cyclic], f'Convolutions', vmax=8)
print(nx.info(G))

Amoung the cyclic boards we can actually see a [Glider](https://en.wikipedia.org/wiki/Glider_(Conway%27s_Life))! This was not possible in 4x4 board, so 5x5 seems to represent more generic case.

##### What is really fascinating about 5x5 cyclic boards is that each of them can be "tiled up" to form 25x25 board which will be also cyclic! (with same peroid)

In [ ]:
plot_3d(board[cyclic], f'Some cyclic 25x25 boards', tile=(5,5))

Now our graph is directed and acyclic and we can start extracting longest passes from it!

In [ ]:
mad0 = list(nx.isolates(G))
for node in mad0:
    G.remove_node(node)

with tqdm(total=nx.number_of_nodes(G)) as pbar:
    path = nx.dag_longest_path(G)
    while len(path) > 1:
        for node in path:
            G.remove_node(node)
        isolates = list(nx.isolates(G))
        for node in isolates:
            G.remove_node(node)
        mad0 = mad0 + path[:1] + isolates
        pbar.update(len(path) + len(isolates))
        plot_3d(board[path], f'{len(path)}-step path', has_mad=True)
        #plot_3d(conv_board[path], f'Convolutions', vmax=8)
        path = nx.dag_longest_path(G)
print(nx.info(G))

As aresult we can see, which parent state we can choose for every non-cyclic state to be able to get as deep as MAD steps into the past. (Output not always correspond to actual parent, but rather equal state that can be obtained from exact parent by translations and flips)

In [ ]:
#L = (len(mad0) + 99) // 100
#for l in tqdm(range(L), total=L):
#    plot_3d(board[mad0[l*100:(l+1)*100]], f'MAD=0 boards, part {l}')